In [14]:
# Import the relevant libraries
from __future__ import division
import pandas as pd
import json
import time
from datetime import datetime, timedelta
import os
import folium
import csv

In [15]:
# working on a one file data
folder="./data/crowdsource-internal/"

# msisdn ='6588769918'
log1 ='6588769918_20170210_210100.log'#for exploring only

log2 ='6588769918_20170210_220100.log'


In [16]:
f = open(os.path.join(folder,log1))
data1=json.load(f)
f.close()

In [17]:
f = open(os.path.join(folder,log2))
data2=json.load(f)
f.close()

In [18]:
data_key =data1.keys()
data_key

[u'msisdn',
 u'locations',
 u'appver',
 u'osver',
 u'model',
 u'trips',
 u'manufacturer']

In [19]:
data1['msisdn']

u'6588769918'

In [20]:
data1['locations'][:2]

[{u'acc': u'22.817',
  u'cid': u'20483393',
  u'lat': u'1.4042991',
  u'lon': u'103.9053817',
  u'signal': u'-93',
  u'ts': u'1486756802106'},
 {u'acc': u'46.0',
  u'cid': u'20483393',
  u'lat': u'1.4041914',
  u'lon': u'103.9054802',
  u'signal': u'-93',
  u'ts': u'1486756805146'}]

In [21]:
# Preprosessing the epoch time
def convert_epoch_time(t):
    # Convert epoch time to utc time
    # PARAMS: t is the epoch time either in seconds or milliseconds. Type : str
    # Output : t in datetime.datetime format


    if len(t)> 12: # t is in milliseconds
        date_time = time.gmtime(int(t)/1000)
    else:
        date_time = time.gmtime(int(t))
        
    # convert to datetime.datetime format
    return datetime(*date_time[:6])
    



In [24]:
# Testing to convert epoch time
x='1486756802106' #in milliseconds

print convert_epoch_time(x)


2017-02-10 20:00:02


In [27]:
df_details1 = pd.DataFrame.from_records(data1['locations'],index='ts')
#print df_details[:4]
# Convert the index of the dataframe to datetime.datetime format

df_details1.index = df_details1.index.map(convert_epoch_time)

df_details1[:5]


,acc,cid,lat,lon,signal
2017-02-10 20:00:02,22.817,20483393,1.4042991,103.9053817,-93
2017-02-10 20:00:05,46.0,20483393,1.4041914,103.9054802,-93
2017-02-10 20:00:22,22.824,20483393,1.4042991,103.9053817,-93
2017-02-10 20:00:24,30.0,20483393,1.4041913,103.90548,-93
2017-02-10 20:00:42,22.84,20483393,1.4042991,103.9053817,-93


In [35]:
# add the other columns to the dataframe


In [28]:
df_details2 = pd.DataFrame.from_records(data2['locations'],index='ts')
#print df_details[:4]
# Convert the index of the dataframe to datetime.datetime format

df_details2.index = df_details2.index.map(convert_epoch_time)

df_details2[:5]

,acc,cid,lat,lon,signal
2017-02-10 21:00:05,22.816,20483393,1.4042991,103.9053817,-93
2017-02-10 21:00:15,58.0,20483393,1.4040958,103.9055678,-93
2017-02-10 21:00:19,22.821,20483393,1.4042991,103.9053817,-93
2017-02-10 21:00:32,63.0,20483393,1.4040994,103.9055713,-93
2017-02-10 21:00:35,22.823,20483393,1.4042991,103.9053817,-93


In [29]:
#combine both the dataframes
df_details1.append(df_details2)


,acc,cid,lat,lon,signal
2017-02-10 20:00:02,22.817,20483393,1.4042991,103.9053817,-93
2017-02-10 20:00:05,46.0,20483393,1.4041914,103.9054802,-93
2017-02-10 20:00:22,22.824,20483393,1.4042991,103.9053817,-93
2017-02-10 20:00:24,30.0,20483393,1.4041913,103.90548,-93
2017-02-10 20:00:42,22.84,20483393,1.4042991,103.9053817,-93
2017-02-10 20:01:01,75.0,20483393,1.4041913,103.9054802,-93
2017-02-10 20:01:36,22.828,20483393,1.4042986,103.9054723,-93
2017-02-10 20:01:38,30.0,20483393,1.4041916,103.9054802,-93
2017-02-10 20:01:55,22.053,20483393,1.4042986,103.9054723,-93
2017-02-10 20:02:03,48.0,20483393,1.4041916,103.9054801,-93


In [34]:
for x in data_key[2:]:
    print x
    print data1[x]
    print '_______'

appver
1.0
_______
osver
4.4.4
_______
model
R7f
_______
trips
[]
_______
manufacturer
OPPO
_______


In [33]:
folder ='./data/crowdsource-internal/'
os.listdir(folder)[:10]

['650000000000009_20170209_100100.log',
 '650000000000009_20170209_140136.log',
 '65111111_20170213_180944.log',
 '6581817105_20170210_130100.log',
 '6581817105_20170210_180349.log',
 '6581817105_20170210_220348.log',
 '6581817105_20170211_070104.log',
 '6581817105_20170211_090348.log',
 '6581817105_20170211_110347.log',
 '6581817105_20170211_130100.log']